In [1]:
%load_ext lab_black

In [2]:
%cd ..

/home/shim/shim/dev/motion-keypoint


In [3]:
import argparse
import json
import math
import sys

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import KFold
from torch import nn, optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from tqdm import tqdm

import networks
import options
import utils
from datasets import KeypointDataset, TestKeypointDataset, get_pose_datasets
from error_list import error_list
from losses import JointMSELoss, KeypointLoss, KeypointRMSE

---

## 하이퍼 파라미터

In [4]:
config = options.load_config("config/hrnet-w48-inference.yaml")

[2021-03-23 10:44:47  INFO] 학습 시작
[2021-03-23 10:44:47  INFO] pose_model: HRNet-W48
[2021-03-23 10:44:47  INFO] result_dir: results/hrnet-train
[2021-03-23 10:44:47  INFO] data_dir: data/ori
[2021-03-23 10:44:47  INFO] lr: 0.0001
[2021-03-23 10:44:47  INFO] batch_size: 10
[2021-03-23 10:44:47  INFO] start_epoch: 1
[2021-03-23 10:44:47  INFO] SAM: False
[2021-03-23 10:44:47  INFO] folds: [1, 2, 3, 4, 5]
[2021-03-23 10:44:47  INFO] checkpoints: [None, None, None, None, None]
[2021-03-23 10:44:47  INFO] padding: 20
[2021-03-23 10:44:47  INFO] add_joint_loss: False
[2021-03-23 10:44:47  INFO] debug: False
[2021-03-23 10:44:47  INFO] step1_epoch: 5
[2021-03-23 10:44:47  INFO] step2_epoch: 10
[2021-03-23 10:44:47  INFO] step3_epoch: 100
[2021-03-23 10:44:47  INFO] input_width: 576
[2021-03-23 10:44:47  INFO] input_height: 768
[2021-03-23 10:44:47  INFO] uid: HRNet_W48
[2021-03-23 10:44:47  INFO] seed: 20210309
[2021-03-23 10:44:47  INFO] inference: True


---

## 데이터셋 준비

In [5]:
dl_train, dl_valid, dl_test = get_pose_datasets(config, 1)

---

## 모델, 데이터셋 생성

In [6]:
import importlib

In [7]:
main = importlib.import_module("main-hrnet-train")

In [8]:
trainer = main.PoseTrainer(config, 1, "results/hrnet-train/ckpt-HRNet_W48_1.pth")

Load pretrained results/hrnet-train/ckpt-HRNet_W48_1.pth


---

## 검증

### Validation에서 1epoch 돌려보기

In [9]:
trainer.evaluate(trainer.dl_valid, "results/hrnet-example/valid_1")

100%|█████████████████████████████| 819/819 [01:03<00:00, 12.85it/s, loss: 1.867542, rmse: 3.408305]


(3.4107230334747127, 10.64324868306881)